In [1]:
import tensorflow as tf
import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline

from random import randint

from load_data import gen_data

In [2]:
tf.reset_default_graph()

num_steps = 25


input_num = 5
output_num = 1

state_size = 20
learning_rate = 0.1
num_layers = 4

x = tf.placeholder(tf.float32, [None, num_steps,input_num], name='input_placeholder')
y = tf.placeholder(tf.float32, [None], name='labels_placeholder')

#rnn_inputs = tf.one_hot(x, num_classes)
rnn_inputs = x

In [3]:
#cell = tf.contrib.rnn.LSTMCell(state_size)
cells = []
for i in range(0, num_layers):
    
    cell = tf.contrib.rnn.GRUCell(state_size )
    cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=1.0-0.04)
    cells.append(cell)
    
mul_cell = tf.nn.rnn_cell.MultiRNNCell(cells)

rnn_outputs, final_state = tf.nn.dynamic_rnn(mul_cell, rnn_inputs, dtype=tf.float32)


In [4]:
rnn_outputs_flat = tf.reshape(rnn_outputs,[tf.shape(rnn_outputs)[0],rnn_outputs.shape[1]*rnn_outputs.shape[2]])

logits = tf.squeeze(tf.layers.dense(rnn_outputs_flat,output_num),axis=1)

loss = tf.reduce_mean(tf.square(y - logits))
train_step = tf.train.AdagradOptimizer(learning_rate).minimize(loss)

In [5]:
batch_size = 5
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for i,(x_,y_) in enumerate(gen_data(["London","Calgary"],['2013','2014'],
                                    batch_size,num_steps)):
    if(y_.shape[0] == 0):
        continue
    #dy = y_[:][-1]
    _,loss_,pred = sess.run([train_step,loss,logits], feed_dict = {x : x_, y: y_})

    if(i%50 == 0):
        print("Iteration : {}  loss : {}  \nreal : {}   \npred : {}\n".format(i, loss_, y_[0:3],pred[0:3]))


London Energy Balance 5min 2013
London Water Balance 5min 2013
London Lysimeters kg 5min2013


/notebooks/Recurrent/load_data.py:68: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...)..apply(<func>)
  'Wind': "mean",'Rain': "sum",'Lysimeter': "mean"})


Iteration : 0  loss : 324.990966796875  
real : [ 18.10318907  18.0803221   18.05376884]   
pred : [-0.00965422  0.09441379  0.05255085]

Iteration : 50  loss : 0.1287166327238083  
real : [ 18.95956629  18.78911441  18.79060037]   
pred : [ 19.10344124  18.87862778  19.53871536]

Iteration : 100  loss : 8.179980278015137  
real : [ 13.82288851  17.1314449   17.25216533]   
pred : [ 14.2445488   13.91338158  14.00990868]

Iteration : 150  loss : 0.24739137291908264  
real : [ 14.09602563  14.08808752  14.02177634]   
pred : [ 13.85900879  13.26789665  14.16982079]

Iteration : 200  loss : 0.18684613704681396  
real : [ 15.47021976  15.47844418  15.46682955]   
pred : [ 14.96752644  15.43684769  15.69403076]

Iteration : 250  loss : 0.11952874809503555  
real : [ 20.77658503  20.71489549  20.7030837 ]   
pred : [ 20.94754982  21.18831062  20.21839142]

Calgary Energy Balance 5min 2013
Calgary Water Balance 5min 2013
Calgary Lysimeters kg 5min2013
Iteration : 300  loss : 0.60545015335083

/usr/local/lib/python3.5/dist-packages/pandas/core/indexes/base.py:2245: FutureWarning: In the future, 'NAT < x' and 'x < NAT' will always be False.
  self._values[0] < other_diff[0]


Iteration : 400  loss : 0.21118824183940887  
real : [ 16.97133426  16.89255168  16.870072  ]   
pred : [ 17.2852993   17.58287811  17.02846909]

Iteration : 450  loss : 0.14036791026592255  
real : [ 16.51546097  16.34446267  16.2462208 ]   
pred : [ 16.86952782  16.38832664  16.32691193]

Iteration : 500  loss : 0.18686100840568542  
real : [ 17.12070126  17.1132969   17.10471418]   
pred : [ 16.33442497  16.70874214  16.8430481 ]

Iteration : 550  loss : 0.16016548871994019  
real : [ 16.92803308  16.92286136  16.9172299 ]   
pred : [ 16.88837624  17.45237541  17.58054352]

Calgary Energy Balance 5min 2014
Calgary Water Balance 5min 2014
Calgary Lysimeters kg 5min2014
Iteration : 600  loss : 0.09306290000677109  
real : [ 14.21298774  14.18440868  14.28919475]   
pred : [ 14.52367878  14.60571766  14.32692242]

Iteration : 650  loss : 0.13163089752197266  
real : [ 13.03580086  12.9753498   12.93583251]   
pred : [ 12.8925333   13.33928871  13.58264256]

Iteration : 700  loss : 0.08

In [17]:
for i,(x_,y_) in enumerate(gen_data(["Halifax"],['2013','2014'],
                                    batch_size,num_steps)):
    if(x_.shape[0] == 0):
        continue
        
    pred = sess.run([logits], feed_dict = {x : x_})
    
    print("Iteration : {}  \nreal : {}   \npred : {}\n".format(i, y_[0:3],pred[0][0:3]))
    if(i > 20):
        break

Halifax Energy Balance 5min 2013
Halifax Water Balance 5min 2013
Halifax Lysimeters kg 5min2013


/notebooks/Recurrent/load_data.py:68: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...)..apply(<func>)
  'Wind': "mean",'Rain': "sum",'Lysimeter': "mean"})


Iteration : 0  
real : [ 13.41168287  13.42844894  13.43358182]   
pred : [ 13.08285999  12.87775517  12.96861744]

Iteration : 1  
real : [ 13.62735885  13.67614649  13.70113795]   
pred : [ 13.13676548  13.04057121  13.57820034]

Iteration : 2  
real : [ 13.91917466  13.97809281  14.02547151]   
pred : [ 13.30455208  13.45019054  13.52787685]

Iteration : 3  
real : [ 14.07297163  13.92831413  13.99635657]   
pred : [ 13.47914982  13.07739544  13.00155067]

Iteration : 4  
real : [ 13.85357935  13.80548777  13.80378317]   
pred : [ 13.33316994  12.90641499  12.99451637]

Iteration : 5  
real : [ 13.65246677  13.6907942   13.5972057 ]   
pred : [ 13.19068432  13.50811768  13.37528992]

Iteration : 6  
real : [ 13.55571162  13.48338298  12.99125408]   
pred : [ 13.5321331   13.16069984  13.04285431]

Iteration : 7  
real : [ 13.17431336  13.07903623  13.23888355]   
pred : [ 12.66262817  13.15732861  12.83452129]

Iteration : 8  
real : [ 12.7830831   12.70791251  12.82977833]   
pred 

In [10]:
data_x,data_y = next(gen_data(["Halifax"],['2013','2014'],
                                    batch_size,num_steps))


Halifax Energy Balance 5min 2013
Halifax Water Balance 5min 2013
Halifax Lysimeters kg 5min2013


/notebooks/Recurrent/load_data.py:68: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...)..apply(<func>)
  'Wind': "mean",'Rain': "sum",'Lysimeter': "mean"})


In [13]:
print(data_y[0:3])

[ 13.41168287  13.42844894  13.43358182]


In [16]:
print(pred[0][0:3])

[ 12.81722736  13.56282902  13.04187489]
